In [1]:
import numpy as np
# import pandas as pd

In [13]:
def objective_function(location):
    return np.sum(location**2)

def denorm(data, domain):
    domain.sort()
    return data * (domain[:, 1] - domain[:, 0]) + domain[:, 0] # data * (max - min) + min

In [14]:
class Property:
    def __init__(self, location, objective_function, domain, price=None, demand=None, supply=None):
        self.location = location
        self.objective_function = objective_function
        self.domain = domain
        self.price = price
        self.demand = demand
        self.supply = supply

    def value(self):
        return self.objective_function(denorm(self.location, self.domain))

In [40]:
class Demander:
    def __init__(self, current_location, objective_function,
                 friend=None, best_location=None, next_location=None, purchase_power=None):
        self.current_location = current_location
        if best_location is None:
            self.best_location = current_location
        else:
            self.best_location = best_location
        if next_location is None:
            self.next_location = current_location
        else:
            self.next_location = next_location
        self.objective_function = objective_function
        if friend is None:
            self.friend = []
        else:
            self.friend = friend
        self.friend = friend
        if purchase_power is None:
            self.purchase_power = np.random.rand()

    def cost_function(self):
        return self.objective_function(self.current_location)

    def add_friend(self, demander_friend):
        self.friend.append(demander_friend)

    # def move(self):
    #     self.current_location = self.next_location
    #     if self.cost_function() > self.objective_function(self.best_location):
    #         self.best_location = self.current_location

In [31]:
location = np.random.uniform(low=-1, high=1, size=(2,))
dimension = len(location)

demander = Demander(current_location=location, objective_function=objective_function)

print(demander.cost_function())


0.5395308189421046


In [34]:
location = np.random.uniform(low=-1, high=1, size=(2,))

demander_1 = Demander(current_location=location, objective_function=objective_function)

In [35]:
demander_1.add_friend(demander)

In [38]:
print(demander_1.friend[0].current_location)


[-0.10493076 -0.72699405]


In [20]:
print(demander_1.location)

[-0.42455397 -0.47852641]


In [17]:
d1 = Property(np.array([0.1, 0.5]), objective_function, domain=np.array([[-65, 65], [-1, 1]]))

In [18]:
d1.value()

2704.0